# Stage 1 – Baseline & Classical Models
**Polymer–Solvent Solubility Prediction**

## Notebook Goals  
1. Explore & tidy the dataset  
2. Train a variety of classical scikit‑learn regressors **(Linear → XGBoost)**, explaining the intuition behind each.  
3. Fit a vanilla PyTorch MLP and compare results.  
4. Record metrics for later stages.


### 0 · Environment & Imports

In [ ]:
# %% [markdown]
"""Install missing deps (Colab)"""
# !pip install --quiet pandas numpy scikit-learn xgboost torch torchvision torchaudio matplotlib seaborn tqdm


In [ ]:
import random, math, pathlib
import numpy as np; import pandas as pd
import matplotlib.pyplot as plt; import seaborn as sns
from pathlib import Path
from tqdm.auto import tqdm

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import metrics

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

from xgboost import XGBRegressor

import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED); random.seed(RANDOM_SEED); torch.manual_seed(RANDOM_SEED)


### 1 · Data Loading

In [ ]:
DATA_DIR = Path('..') / 'data'  # adjust
df_exp     = pd.read_csv(DATA_DIR / 'experimental_dataset.csv')
df_poly    = pd.read_csv(DATA_DIR / 'list_of_polymers.csv')
df_solv    = pd.read_csv(DATA_DIR / 'list_of_solvents.csv')
df_poly_mw = pd.read_csv(DATA_DIR / 'polymer_mass.csv')
df_solv_mw = pd.read_csv(DATA_DIR / 'solvent_mass.csv')
df_solv_mac= pd.read_csv(DATA_DIR / 'solvent_macro_features.csv')

print('experimental_dataset:', df_exp.shape)


In [ ]:
full_df = (df_exp
           .merge(df_poly,    on='polymer_id',  how='left')
           .merge(df_solv,    on='solvent_id',  how='left')
           .merge(df_poly_mw, on='polymer_id',  how='left')
           .merge(df_solv_mw, on='solvent_id',  how='left')
           .merge(df_solv_mac,on='solvent_id',  how='left'))
print(full_df.shape)
full_df.head()


### 2 · Exploratory Data Analysis

In [ ]:
TARGET_COL = 'solubility'  # adjust
full_df.isna().sum().sort_values(ascending=False).head(10)


In [ ]:
sns.histplot(full_df[TARGET_COL], kde=True)
plt.title('Target distribution'); plt.show()


### 3 · Feature Engineering & Pre‑processing

In [ ]:
num_cols = [c for c in full_df.columns if full_df[c].dtype != 'object' and c not in ['polymer_id','solvent_id', TARGET_COL]]
cat_cols = [c for c in full_df.columns if full_df[c].dtype == 'object']

preprocessor = ColumnTransformer([
    ('num', Pipeline([('scaler', StandardScaler())]), num_cols),
    ('cat', Pipeline([('onehot', OneHotEncoder(handle_unknown='ignore'))]), cat_cols)])


### 4 · Dataset Split

In [ ]:
train_val_df, test_df = train_test_split(full_df, test_size=0.15, random_state=RANDOM_SEED)
train_df, val_df     = train_test_split(train_val_df, test_size=0.15, random_state=RANDOM_SEED)
print(len(train_df), len(val_df), len(test_df))

X_train, y_train = train_df.drop(columns=[TARGET_COL]), train_df[TARGET_COL]


## 5 · Model Zoo – Train & Explain

#### 5.1 Linear Regression – baseline, interpretable

In [ ]:
pipe_lr = Pipeline([('pre', preprocessor), ('lr', LinearRegression())])
lr_rmse = -cross_val_score(pipe_lr, X_train, y_train, scoring='neg_root_mean_squared_error', cv=5).mean()
print('LinearRegression CV RMSE:', lr_rmse)


#### 5.2 Ridge Regression – shrink coefficients

In [ ]:
for a in [0.01,0.1,1,10]:
    score = -cross_val_score(Pipeline([('pre', preprocessor), ('ridge', Ridge(alpha=a))]),
                              X_train, y_train, scoring='neg_root_mean_squared_error', cv=5).mean()
    print(f'alpha={a}: {score:.4f}')


#### 5.3 k‑NN – non‑parametric

In [ ]:
for k in [3,5,10,20]:
    score = -cross_val_score(Pipeline([('pre', preprocessor), ('knn', KNeighborsRegressor(n_neighbors=k))]),
                              X_train, y_train, scoring='neg_root_mean_squared_error', cv=5).mean()
    print(f'k={k}: {score:.4f}')


#### 5.4 Decision Tree – capturing non‑linear splits

In [ ]:
for depth in [None,5,10,20]:
    score = -cross_val_score(Pipeline([('pre', preprocessor), ('dt', DecisionTreeRegressor(max_depth=depth))]),
                              X_train, y_train, scoring='neg_root_mean_squared_error', cv=5).mean()
    print(f'depth={depth}: {score:.4f}')


#### 5.5 Random Forest – ensemble reduces variance

In [ ]:
rf = Pipeline([('pre', preprocessor), ('rf', RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=RANDOM_SEED))])
rf_rmse = -cross_val_score(rf, X_train, y_train, scoring='neg_root_mean_squared_error', cv=5).mean()
print('RandomForest CV RMSE:', rf_rmse)


#### 5.6 Gradient Boosting – sequential error correction

In [ ]:
gb = Pipeline([('pre', preprocessor), ('gb', GradientBoostingRegressor(n_estimators=500, learning_rate=0.05))])
gb_rmse = -cross_val_score(gb, X_train, y_train, scoring='neg_root_mean_squared_error', cv=5).mean()
print('GradientBoosting CV RMSE:', gb_rmse)


#### 5.7 Support Vector Regression – kernel trick

In [ ]:
for k in ['linear','rbf']:
    svr = SVR(kernel=k, C=5, epsilon=0.1)
    score = -cross_val_score(Pipeline([('pre', preprocessor), ('svr', svr)]),
                              X_train, y_train, scoring='neg_root_mean_squared_error', cv=5).mean()
    print(f'SVR-{k}: {score:.4f}')


#### 5.8 XGBoost – powerful boosting with regularisation

In [ ]:
xgb = XGBRegressor(n_estimators=800, max_depth=6, learning_rate=0.05, subsample=0.8,
                  colsample_bytree=0.8, objective='reg:squarederror', random_state=RANDOM_SEED, n_jobs=-1)
pipe_xgb = Pipeline([('pre', preprocessor), ('xgb', xgb)])
xgb_rmse = -cross_val_score(pipe_xgb, X_train, y_train, scoring='neg_root_mean_squared_error', cv=5).mean()
print('XGBoost CV RMSE:', xgb_rmse)


## 6 · Vanilla PyTorch MLP

In [ ]:
preprocessor.fit(X_train)
X_train_np = preprocessor.transform(X_train).astype(np.float32)
y_train_np = y_train.values.astype(np.float32).reshape(-1,1)

class SolvDataset(Dataset):
    def __init__(self,X,y): self.X=torch.tensor(X); self.y=torch.tensor(y)
    def __len__(self): return len(self.X)
    def __getitem__(self,idx): return self.X[idx], self.y[idx]

dloader = DataLoader(SolvDataset(X_train_np,y_train_np), batch_size=256, shuffle=True)

class MLP(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(in_dim,256), nn.ReLU(), nn.BatchNorm1d(256),
                                 nn.Linear(256,128), nn.ReLU(), nn.BatchNorm1d(128),
                                 nn.Linear(128,1))
    def forward(self,x): return self.net(x)

model = MLP(X_train_np.shape[1]); opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss();

for epoch in range(50):
    model.train(); total=0
    for xb,yb in dloader:
        opt.zero_grad(); pred=model(xb); loss=loss_fn(pred,yb); loss.backward(); opt.step(); total+=loss.item()*len(xb)
    if (epoch+1)%10==0:
        print(f'Epoch {epoch+1:02d}  Train MSE={total/len(dloader.dataset):.4f}')


### 7 · Wrap‑up
Summarise RMSEs from each model above, draw conclusions, and plan Stage 2.